# IMPORTS E CONFIGURAÇÕES

## Imports

In [2]:
import os
import sys
import pickle
import pandas as pd
import numpy as np
import requests

import xgboost as xgb
from io import StringIO
from flask import Flask, request, Response

# Adiciona o diretório raiz do projeto ao path do Python
sys.path.append('../')
from api.rossmann.Rossmann import Rossmann

## Configurações

In [3]:
pd.set_option('display.max_columns', None)

# LOAD DATA

## Carregar dados

In [4]:
# --- Carregar os dados brutos que o modelo nunca viu ---
df_test = pd.read_csv('../data/raw/test.csv', low_memory=False)
df_store = pd.read_csv('../data/raw/store.csv', low_memory=False)
df_raw = pd.merge(df_test, df_store, on='Store', how='left')

## Carregar modelo e encoders manualmente 

In [5]:
MODEL_PATH = '../model/modeling/'
SCALERS_PATH = '../model/pre-processing/scalers.pkl'    # dicionário com scalers
ENCODERS_PATH = '../model/pre-processing/encoders.pkl'  # dicionário com encoders
FEATURES_PATH = '../model/pre-processing/list_features_selected.pkl'

# Carregar modelo
model = xgb.XGBRegressor()
# The path should be relative to BASE_PATH, without a leading slash.
model.load_model(os.path.join(MODEL_PATH, "model_xgb_final.ubj"))

# Carregar scalers
with open(SCALERS_PATH, 'rb') as f:
    scalers = pickle.load(f)

# Carregar encoders
with open(ENCODERS_PATH, 'rb') as f:
    encoders = pickle.load(f)

# Carregar features selecionadas
with open(FEATURES_PATH, 'rb') as f:
    features_selected = pickle.load(f)

# DEPLOY

## Classe Rossmann

In [6]:
# Criar a instância do pipeline Rossmann
# que será usado para fazer previsões
pipeline = Rossmann(
    model=model,
    scalers=scalers,
    encoders=encoders,
    features_selected=features_selected
)

# Escolher as lojas para predição
df_sample = df_raw[df_raw['Store'].isin( [24, 12, 22] )]

2025-08-15 11:06:02,637 - INFO - Rossmann initialized with preloaded objects.


### Realizar pré-processamento

In [7]:
# 3. Aplicar o pipeline de pré-processamento
df_prepared = pipeline.preprocess(df_sample.copy())

### Executar o modelo de previsão de vendas

In [8]:
# 4. Fazer a previsão
df_response_json = pipeline.get_prediction(df_prepared)

### Ver resultado das previsões de vendas

In [9]:
# 5. Converter o JSON de resposta para um DataFrame para visualização
#    2. Usar StringIO para evitar o FutureWarning
df_response = pd.read_json(StringIO(df_response_json))

# 6. Visualizar o resultado com a coluna 'Prediction'
df_prediction = df_response[['Store', 'Prediction']].groupby('Store').sum().reset_index()

for i in range(len( df_prediction )):
    print('Store Number {} will sell R${:,.2f} in the next 6 weeks'.format(
        df_prediction.loc[i, 'Store'],
        df_prediction.loc[i, 'Prediction'] ) )

Store Number 12 will sell R$338,412.86 in the next 6 weeks
Store Number 22 will sell R$187,366.61 in the next 6 weeks
Store Number 24 will sell R$323,053.97 in the next 6 weeks


## Chamada da API

> Execute o arquivo handler.py, através do comando abaixo, pelo terminal para ativar o serviço e poder testar a API localmente.
>
> ```python handler.py ```

### Definir variáveis na API

In [10]:
# URL de desenvolvimento
# url = 'http://0.0.0.0:8080/rossmann/predict'

# URL de produção
url = 'https://rossmann-sales-forecast-flnh.onrender.com/rossmann/predict'

header = {
    'Content-type': 
    'application/json' 
}
data = df_sample.to_json(orient="records", date_format="iso")

### Executar a API

In [11]:
r = requests.post(
    url, 
    data=data, 
    headers=header
)
print('Status Code {}'.format(r.status_code))

Status Code 200


### Visualizar retorno da API

In [12]:
d1 = pd.DataFrame(
    r.json(), 
    columns=r.json()[0].keys()
)

d2 = (
    d1[['Store', 'Prediction']]
    .groupby('Store')
    .sum()
    .reset_index()
)

for i in range(len(d2)):
    print(
        'A loja {} venderá R$ {:,.2f} nas próximas 6 semanas'.format(
        d2.loc[i, 'Store'],
        d2.loc[i, 'Prediction']) 
    )


A loja 12 venderá R$ 338,412.86 nas próximas 6 semanas
A loja 22 venderá R$ 187,366.61 nas próximas 6 semanas
A loja 24 venderá R$ 323,053.97 nas próximas 6 semanas


## Chamada Telegram

### Info about the Bot
'https://api.telegram.org/bot6869086822:AAEnR8Rl9yIiT-MUVwCkOstK_KpkEvck9fo/getMe'

### Get Updates
'https://api.telegram.org/bot6869086822:AAEnR8Rl9yIiT-MUVwCkOstK_KpkEvck9fo/getUpdates'

### Send Message
https://api.telegram.org/bot6869086822:AAEnR8Rl9yIiT-MUVwCkOstK_KpkEvck9fo/sendMessage?chat_id=1090663968&text=Hi Ale, I am doing good, tks!

### Delete Webhook
curl https://api.telegram.org/bot6869086822:AAEnR8Rl9yIiT-MUVwCkOstK_KpkEvck9fo/deleteWebhook

### Get Webhook Info
curl https://api.telegram.org/bot6869086822:AAEnR8Rl9yIiT-MUVwCkOstK_KpkEvck9fo/getWebhookInfo

### Set Webhook

#### Pelo Browser:
https://api.telegram.org/bot6869086822:AAEnR8Rl9yIiT-MUVwCkOstK_KpkEvck9fo/setWebhook?url=https://rossmann-telegram-bot-92ce.onrender.com

#### Pelo Terminal
```
curl -F "url=https://rossmann-telegram-bot.onrender.com/" https://api.telegram.org/bot6869086822:AAEnR8Rl9yIiT-MUVwCkOstK_KpkEvck9fo/setWebhook
```
```
curl -F "url=https://5536b31df9cdf6.lhr.life/" https://api.telegram.org/bot6869086822:AAEnR8Rl9yIiT-MUVwCkOstK_KpkEvck9fo/setWebhook
```
